<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [ ]:
#Creae Dataset

import os
from os.path import exists, join, basename, splitext
import json
import numpy as np



json_path = "/content.gdrive/My Drive/Try/data.json"
#json_path = "/content/data.json"
# dictionary to store mapping, labels, and Keypoints
data = {
    "labels":[],
    "keypoints":[]
}

PATH = "/content.gdrive/My Drive/Try"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt

    folder_dir = Path_Pic + '/exp'
    %cd /content/openpose/build/examples/tutorial_api_python
    !python3 04_keypoints_from_images.py --image_dir '{folder_dir}' --keypoint_scale 3 --no_display true

    with open(Path_Pic + '/exp/keypoints.json', "r") as fp:
        keypoints_json = json.load(fp)

    data["keypoints"].append(keypoints_json)

    if l == 'NoTelemark':
        data["labels"].append(0)
    if l == 'Telemark':
        data["labels"].append(1)    

    #with open(Path_Pic + '/exp/keypoints.txt', "r") as keypointfile:
        #keypoints = keypointfile.read()
    #data["keypoints"].append(keypoints)


with open(json_path, "w") as fp:
  json.dump(data, fp, indent=4)


In [10]:
#Creating binary classification model
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.utils import shuffle


model = Sequential([
    Dense(units=2500, input_shape=(1575,), activation='relu'),
    Dense(units=1250, activation='relu'),
    Dense(units=250, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.summary()


with open('/content.gdrive/MyDrive/Try/data_try1.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
X = np.array(data["keypoints"])
#X = np.array(data["mfcc"])
y = np.array(data["labels"])

X,y = shuffle(X,y)

print(X[0])
print(X.shape)
print(y[0])
print(y.shape)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])





Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 2500)              3940000   
                                                                 
 dense_21 (Dense)            (None, 1250)              3126250   
                                                                 
 dense_22 (Dense)            (None, 250)               312750    
                                                                 
 dense_23 (Dense)            (None, 2)                 502       
                                                                 
Total params: 7,379,502
Trainable params: 7,379,502
Non-trainable params: 0
_________________________________________________________________
[0.56830007 0.38073403 0.58075005 ... 0.         0.         0.        ]
(104, 1575)
0
(104,)


In [11]:
model.fit(x=X, y=y, batch_size=10, validation_split = 0.1, epochs=10, shuffle=True, verbose = 2)

Epoch 1/10
10/10 - 2s - loss: 0.7425 - accuracy: 0.5161 - val_loss: 0.7180 - val_accuracy: 0.7273 - 2s/epoch - 160ms/step
Epoch 2/10
10/10 - 1s - loss: 0.5539 - accuracy: 0.7097 - val_loss: 1.1046 - val_accuracy: 0.2727 - 582ms/epoch - 58ms/step
Epoch 3/10
10/10 - 1s - loss: 0.5761 - accuracy: 0.6882 - val_loss: 0.8720 - val_accuracy: 0.6364 - 510ms/epoch - 51ms/step
Epoch 4/10
10/10 - 1s - loss: 0.5661 - accuracy: 0.7097 - val_loss: 1.0104 - val_accuracy: 0.6364 - 508ms/epoch - 51ms/step
Epoch 5/10
10/10 - 0s - loss: 0.5178 - accuracy: 0.7742 - val_loss: 0.9155 - val_accuracy: 0.4545 - 494ms/epoch - 49ms/step
Epoch 6/10
10/10 - 0s - loss: 0.3894 - accuracy: 0.7849 - val_loss: 1.1065 - val_accuracy: 0.2727 - 485ms/epoch - 48ms/step
Epoch 7/10
10/10 - 0s - loss: 0.3053 - accuracy: 0.9570 - val_loss: 1.1581 - val_accuracy: 0.2727 - 498ms/epoch - 50ms/step
Epoch 8/10
10/10 - 0s - loss: 0.2864 - accuracy: 0.9032 - val_loss: 1.3386 - val_accuracy: 0.2727 - 488ms/epoch - 49ms/step
Epoch 9/10

In [12]:
predictions = model.predict(X, batch_size = 10, verbose =0)

In [13]:
for i in predictions:
  print(i)

[0.9966413  0.00335868]
[0.9894678  0.01053227]
[0.42375273 0.5762472 ]
[0.0628267  0.93717325]
[0.07208315 0.9279169 ]
[0.98811054 0.01188947]
[0.9671194  0.03288059]
[0.9947103 0.0052897]
[0.6417022  0.35829785]
[0.9890337  0.01096634]
[0.32248393 0.67751604]
[0.09596359 0.9040364 ]
[0.9883642  0.01163575]
[0.9706898  0.02931018]
[0.03246811 0.9675319 ]
[0.05246657 0.9475334 ]
[0.97902757 0.02097246]
[0.1881209  0.81187916]
[0.98745745 0.01254252]
[0.98552644 0.01447354]
[0.9670702  0.03292977]
[0.97820836 0.02179164]
[0.98528093 0.01471903]
[0.9935361  0.00646386]
[0.08106289 0.9189371 ]
[0.28106508 0.7189349 ]
[0.9839663  0.01603364]
[0.97153497 0.02846503]
[0.99044764 0.0095524 ]
[0.9540058  0.04599427]
[0.97357935 0.02642069]
[0.80394685 0.19605318]
[0.23021293 0.769787  ]
[0.9034219  0.09657812]
[0.9798187  0.02018129]
[0.95201176 0.04798827]
[0.5870854 0.4129145]
[0.9768312  0.02316881]
[0.9893294  0.01067058]
[0.9405359  0.05946401]
[0.9015674  0.09843261]
[0.9806009  0.019399